In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI

from IPython.display import Markdown, display, update_display

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Gemini
openai = OpenAI()
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
# Let's make a conversation between GPT-4.1-mini and Gemini-2.0-flash
# We're using cheap versions of models so the costs will be minimal

game = "Santorini"
no_questions = 3

gpt_model = "gpt-4o-mini"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are a boardgame journalist. \
You tend to be objective and ask right questions to get to the core of the boardgame mechanics, \
visual appeal and time to setup the game. Your goal is to ask the right questions to get the best possible review of the board game." \
"You ask one question at a time and wait for the other person to answer. \
You do not answer your own questions. You always try to build on the previous answer."

gemini_system = "You are a boardgame critique; \
you tend to objectively analyze everything when it comes to a board game gameplay, visual appeal and time to setup the game. \
Your goal is to provide constructive criticism so the board gaming community can benefit from these insights." \
"You answer one question at a time and wait for the other person to ask the next question. \
You do not ask your own questions. You always try to build on the previous question. \
If the other person is very positive, you try to point out flaws in the game. \
If the other person is very negative, you try to point out good aspects of the game."

gpt_messages = [f"I would like to review the board game {game}."]
gemini_messages = [f"Sure, ask me anything about the board game {game}."]


In [ ]:
def call_boardgame_journalist():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_boardgame_critique():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    completion = gemini_via_openai_client.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return completion.choices[0].message.content


In [ ]:
gpt_messages = [f"I would like to review the board game {game}."]
gemini_messages = [f"Sure, ask me anything about the board game {game}."]

print(f"Journalist:\n{gpt_messages[0]}\n")
print(f"Critique:\n{gemini_messages[0]}\n")

for i in range(no_questions):
    gpt_next = call_boardgame_journalist()
    print(f"Journalist:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    gemini_next = call_boardgame_critique()
    print(f"Critique:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)